In [ ]:
import yfinance as yf
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
from LoadConfig import load_config  # 심볼 JSON 로딩 함수

def check_batch(symbols: list[str], sleep_per_symbol: float = 0.1) -> list[str]:
    """
    심볼 리스트를 받아 유효한 1분봉 데이터가 있는 종목만 반환합니다.
    """
    valid = []
    for symbol in symbols:
        try:
            df = yf.Ticker(symbol).history(interval="1m", period="1d", prepost=True)
            if not df.empty:
                print(f"[통과] {symbol}")
                valid.append(symbol)
            else:
                print(f"[제외] {symbol}: 데이터 없음")
        except Exception as e:
            print(f"[에러] {symbol}: {e}")
        time.sleep(sleep_per_symbol)  # Yahoo API 과부하 방지용 슬립
    return valid

def run_batch_filtering(
    symbols: list[str],
    batch_size: int = 50,
    max_workers: int = 25,
    sleep_per_symbol: float = 0.0
) -> list[str]:
    """
    심볼 전체를 배치 단위로 나눠서 멀티스레드 처리.
    """
    batches = [symbols[i:i+batch_size] for i in range(0, len(symbols), batch_size)]
    results = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(check_batch, batch, sleep_per_symbol) for batch in batches]

        for future in as_completed(futures):
            try:
                result = future.result()
                results.extend(result)
            except Exception as e:
                print(f"[에러] 배치 처리 실패: {e}")

    return results

if __name__ == "__main__":
    symbols = load_config("symbols_valid.json")
    print(f"[시작] 총 심볼 수: {len(symbols)}")

    valid_symbols = run_batch_filtering(
        symbols=symbols
    )

    print(f"\n[완료] 유효한 심볼 수: {len(valid_symbols)}")
    print(valid_symbols)
